https://www.analyticsvidhya.com/blog/2021/08/python-tutorial-working-with-csv-file-for-data-science/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive/Academic/CSCM35 - Big Data & Data Mining/coursework 2/Dataset.Small" "/content/"
!ln -s "/content/drive/My Drive/Academic/CSCM35 - Big Data & Data Mining/coursework 2/Dataset.Large" "/content/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ln: failed to create symbolic link '/content/Dataset.Small': File exists
ln: failed to create symbolic link '/content/Dataset.Large': File exists


In [ ]:
from itertools import combinations
from collections import Counter
import numpy as np
import csv


In [ ]:
def read_from_file(file_name) :
  csvfile = open(file = file_name, mode = 'r') # open file
  print(type(csvfile))

  csvreader = csv.reader(csvfile, skipinitialspace = True) # csv reader object
  rows = [] # empty list
  for row in csvreader:
    rows.append(row)

  csvfile.close() #' close file
  return rows

def write_to_file(file_name, data) :
  file = open(file = file_name, mode = 'w') # file object

  csvwriter = csv.writer(file) # csv reader object
  for row in data:
    csvwriter.writerow(row)

  file.close() # close file

In [ ]:
transactions = read_from_file('Dataset.Small/GroceryStore.csv') # read transactions or database
write_to_file('Transactions.csv', transactions) # write transactions or database into another file

<class '_io.TextIOWrapper'>


Brute-Force Approach

$
Number\ of\ unique\ items\ =\ d\\
Number\ of\ transactions\ =\ N\\
Average\ width\ of\ a\ transaction=\ w\\ \\
Number\ of\ all\ possible\ combinations\ =\
\displaystyle\sum_{r=1} ^{d} c_{r}\ =\ 2^d - 1\\
Complexity\ =\ O(Nw2^{d})
$

In [ ]:
def brute_force_approach(database, unique_items, min_sup_cnt, item_set_groups):

  for item_cnt in item_set_groups: # loop number of itemset groups
    item_sets = combinations(unique_items, item_cnt) # create combinations for each itemset group
    for item_set in item_sets: # itemset : tuple of items
      support_count = np.count_nonzero([all(item in transaction for item in item_set) for transaction in database])
      print(support_count)
      

transactions = read_from_file('Dataset.Small/GroceryStore.csv') # transactions or database
database = []
for transaction in transactions:
  tr_str = ''.join(transaction)
  database.append(tr_str.split(','))
unique_items = read_from_file('Dataset.Small/Items.txt') # unique items

min_sup_cnt = 2
item_set_groups = [2]
brute_force_approach(database, unique_items[0], min_sup_cnt, item_set_groups)

<class '_io.TextIOWrapper'>
<class '_io.TextIOWrapper'>
2
0
0
0
1
0
0
2
0
1
0
0
0
4
0
1
3
2
1
0
0
0
0
0
0
0
0
0
1
1
4
3
2
1
0
0
0
0
0
0
2
2
4
2
1
0
3
0
0
1
3
2
4
4
2
